In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 数据集的原始路径
dataset_path = 'D:/tester01'

# 创建训练和验证文件夹
train_path = 'D:/tester01/train'
val_path = 'D:/tester01/val'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# 分类别进行划分
categories = ['cataract', 'normal', 'surgery']

for category in categories:
    # 创建类别的训练和验证目录
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(val_path, category), exist_ok=True)

    # 获取所有图片路径
    images = os.listdir(os.path.join(dataset_path, category))
    images = [os.path.join(dataset_path, category, image) for image in images]

    # 划分数据集
    train_images, val_images = train_test_split(images, test_size=0.2) # 这里以80%训练，20%验证的比例划分

    # 移动图片到对应的训练和验证目录
    for image in train_images:
        shutil.move(image, os.path.join(train_path, category))

    for image in val_images:
        shutil.move(image, os.path.join(val_path, category))

# 构建一个简单的CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 假设有三个类别
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 数据增强
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# 准备数据集
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

# 训练模型
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

# 保存训练好的模型
model.save("cat.h5")
